<a href="https://colab.research.google.com/github/markaaronslater/NMT/blob/master/playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2



In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [3]:

!pip install subword-nmt # for segmenting words into subwords
!pip install stanza # for tokenizing corpus and tagging with morphological data
!pip install sacremoses # for detokenizing model predictions
!pip install sacrebleu # for evaluation


     |████████████████████████████████| 235kB 11.5MB/s 
     |████████████████████████████████| 890kB 8.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=20eff72477284e0d88f65b0463ad0bb4a3276c78fd7e3864f1649bde3d84a74b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 71kB 5.3MB/s 


In [4]:

# make sure using GPU
# (Runtime -> Change runtime type -> Hardware accelerator = GPU).
!nvidia-smi


Fri Nov  6 00:38:25 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:

%cd /content/gdrive/My Drive/NMT


/content/gdrive/My Drive/NMT


In [5]:

from src.model_utils import load_pretrained
from src.translate import translate
from src.predict import predict
from src.evaluate import evaluate

import stanza
from subword_nmt.apply_bpe import BPE


In [6]:
# recommended path to project root directory: place cloned NMT folder in 'My Drive' folder of Google Drive account:
path = '/content/gdrive/My Drive/NMT/'
model_name = 'old_replica/' # name of pre-trained model to load
checkpoint_path = path + 'checkpoints/' + model_name
corpus_path = path + 'corpuses/iwslt16_en_de/subword_segmented/'
### ??how to give them access to my pre-trained model?? too large to commit to github??

translator, model_data = load_pretrained(checkpoint_path=checkpoint_path)
#test_batches = model_data["test_batches"]
src_word_to_idx = model_data["src_word_to_idx"]
idx_to_trg_word = model_data["idx_to_trg_word"]


In [7]:

stanza.download(lang='de', processors='tokenize,mwt,pos')
stanza_de_processor = stanza.Pipeline(lang='de', processors='tokenize,mwt,pos', tokenize_no_ssplit=True, tokenize_batch_size=64, mwt_batch_size=200, pos_batch_size=10000)


2020-11-06 00:54:31 INFO: Downloading these customized packages for language: de (German)...
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| pretrain  | gsd     |

2020-11-06 00:54:31 INFO: File exists: /root/stanza_resources/de/tokenize/gsd.pt.
2020-11-06 00:54:31 INFO: File exists: /root/stanza_resources/de/mwt/gsd.pt.
2020-11-06 00:54:31 INFO: File exists: /root/stanza_resources/de/pos/gsd.pt.
2020-11-06 00:54:31 INFO: File exists: /root/stanza_resources/de/pretrain/gsd.pt.
2020-11-06 00:54:31 INFO: Finished downloading models and saved to /root/stanza_resources.
2020-11-06 00:54:31 INFO: Loading these models for language: de (German):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |

2020-11-06 00:54:31 INFO: Use device: gpu
2020-11-06 00:54:31 INFO: Loading: tokenize
2020-11-06 00:54:32 INFO: Loading: mwt
2020-11-06 00:54:32 INFO: Loading

In [8]:
bpe = BPE(open(corpus_path + 'bpe_codes', 'r'), vocab=set(src_word_to_idx))

In [11]:
# a) translation playground:
# place any number of whatever German sentences you want as strings inside following list:
input = ["Dies ist ein deutscher Testsatz. Wird das Modell es erfolgreich übersetzen können?", 
         "Wenn nicht, wird diese Demo nicht sehr beeindruckend sein ...",
         "Ich empfehle, dass Sie zuerst einen englischen Satz erstellen und ihn dann mit Google Translate in Deutsch konvertieren."]

# determined via Google Translate:
sample_targets = ["This is a German test sentence. Will the model be able to translate it successfully?",
                  "If it doesn't, then this demo will not be very impressive...",
                  "I recommend that you first come up with an English sentence, and then use Google Translate to convert it to German."]

translations = translate(input, stanza_de_processor, translator, src_word_to_idx, idx_to_trg_word, bpe, device='cuda:0', bsz=8)
for translation in translations:
    print(translation)

tokenizing, multiword-token-expanding, pos-tagging...
truecasing...
subword segmenting...
converting to batches of indices...
making predictions...
This is a German test. Will the model be able to translate it?
If not, this demo is not very impressive...
I suggest that you're going to make a English sentence, and then you put it with Google Translate in German.


In [ ]:
# optional - if targets are available, evaluate via BLEU metric:
print(evaluate(translations, [sample_targets])

In [ ]:
# b) replicate BLEU score on test set



# download iwslt17 test set
